# 1. Basic Usage

`TriPodPy` employs the `simframe` framework for running scientific simulations. For a detailed description of the usage of `simframe` please have a look at the [Simframe Documentation](https://simframe.rtfd.io/). The `tripod` Simulation class inherits many of its atributes from so for the basic structure looking at the [Dustpy Documentation](https://stammler.github.io/dustpy/) can be helpful.

This notebook demonstrates the basic steps of running a `TriPodPy` simulation:
- Initializing and running the most simple (default) `TriPodPy` model
- Plotting the data
- Resuming simulations from dump files

In [ ]:
import numpy as np

## The Simulation Frame

To set up a model we have to import the `Simulation` class from the `tripod` package.

In [ ]:
from tripod import Simulation

We can now create an instance of this class.

In [ ]:
sim = Simulation()
sim.initialize()

At this stage `sim` is an empty simulation object that controls our simulation.

In [ ]:
sim

All the fields are initialized with `None`. All attributes can be easiliy addressed via e.g.

In [ ]:
sim.gas

## Initializing

We can now initialize the `sim` object with `Simulation.initialize()`. `TriPodPy` will then fill all the fields with default values.

As we can see, the `sim` object has now values assigned to its fields.  
**All quantities are in cgs units.**

In [ ]:
sim.gas

We can also display the full table of contents of the `sim` object.

In [ ]:
sim.toc

The initial condition of the default simulation is printed below. For mean and details about the varibles see Section 4.

In [ ]:
print(Simulation().ini.dust)
print(Simulation().ini.gas)
print(Simulation().ini.grid)
print(Simulation().ini.star)

## Running a Simulation

The simulation is now ready to go. We can start it with ``Simulation.run()``. As the default simulation runs for 100,000 years, this will merely take a few moments.

In [ ]:
sim.writer.overwrite = True

In [ ]:
sim.run()

By default, `TriPodPy` has used the hdf5 writer to save output files to the `data/` directory.

## Plotting

`TriPodPy` is coming with a simple plotting script that can be used to check the status of a simulation.

In [ ]:
from tripod import plot
import dustpy.constants as c

The plotting script does either take the simulation object as argument or a data directory.

If the argument is a simulation object the script is only plotting the current state.

In [ ]:
plot.panel(sim,show_limits=True,ir=20)

The blue and the green lines in the top left panel are analytical estimates for the fragmentation and drift barrier taken from [Birnstiel et al. (2012)](https://doi.org/10.1051/0004-6361/201118136).the middle panel on the top shows the size distribution at the radial index `ir`. the top right shows the mass evolution of dust and gas (if multiple snapshots are provided). the bottom left shows the midplane density of the dust (red) and the power law exponent q (blue). the bottom middle pannel shows the total dust and gas surface desity along with the dust do gas ratio (gray) as a function of radius. If we pass the data directory as argument, we also have access to the time evolution. Furthermore, some plots can be addressed by specifying the time `it`, radial `ir`, or size `ia` index.

In [ ]:
plot.panel("data", it=10, ir=10, ia=20,show_limits=True)

### Dust distribution with TriPoD
`TriPodPy`simulates the evolution of the dust surface density in terms of the two population surface densities $\Sigma_0$ and $\Sigma_1$.

Together, they constitute the total dust density $\Sigma_{tot} = \int \limits_{m\left(s_{min}\right)}^{m\left(s_{max}\right)} \sigma\left(m\right) \mathrm{d}m = \int \limits_{m\left(s_{min}\right)}^{m\left(s_{int}\right)} \sigma\left(m\right) \mathrm{d}m + \int \limits_{m\left(s_{int}\right)}^{m\left(s_{max}\right)} \sigma\left(m\right) \mathrm{d}m = \Sigma_0 + \Sigma_1$.

In [ ]:
SigmaTot = sim.dust.Sigma.sum(-1)
SigmaTot

In order to display the data in a more granular way and compare it to `Dustpy` outputs, we can use the distribution exponents $\xi$ to interpolate the dust surface density over a logarithmic mass grid. The density in the mass interval $\left[m_0, m_1\right]$ is then given by

$\Sigma_{[m_0, m_1]} = \int\limits_{m_0}^{m_1} \sigma\left(m\right) \mathrm{d}m = \Sigma_{tot} \frac{m_1^{\frac{\xi+4}{3}}-m_0^{\frac{\xi+4}{3}}}{m_{max}^{\frac{\xi+4}{3}}-m_{min}^{\frac{\xi+4}{3}}}$ for $\xi_{calc} \neq -4$ and $\Sigma_{tot} \frac{\log(m_1/m_0)}{\log(m_{max}/m_{min})}$ for $\xi_{calc} = -4$.

This calculation is automatically done in the data readout scheme of the plotting routine:

In [ ]:
from tripod.utils import read_data
data = read_data("data")

The plotted dust density distribution is then given by

$\Sigma_\mathrm{d} = \int\limits_0^\infty \sigma \left(m \right) \mathrm{d} \log m$.

In this way the distribution is independent of the mass grid.

The code units `TriPodPy` uses after interpolation of the dust density are $\Sigma_{\mathrm{d},\,i} \equiv \Sigma_\mathrm{d} \left(m_i \right)$ with

$\Sigma_\mathrm{d} = \sum\limits_i \Sigma_{\mathrm{d},\,i}$,

meaning the numerical sum over the mass dimension is the dust surface density.

In [ ]:
data.dust.Sigma_recon.sum(-1)[-1,...]

For a more in depth analysis the internally used sizes and power law exponents can be found in the simulation Object/data under:

power law exponent of the particle size distribution:

In [ ]:
sim.dust.qrec[:5]

the minimum and maximum sizes of the distribution:

In [ ]:
sim.dust.s.min[:5]

In [ ]:
sim.dust.s.max[:5]

the size array with all the sizes used for the evolution of the distribution:

$[a_0, 0.4 * a_1, a_1, 0.4 * a_{max}, a_{max}]$

In [ ]:
sim.dust.a[0,:]

this means all the dust quantities either have shape $(N_r, 2)$ or $(N_r, 5)$ depending if they are defined for the mass bins or the particle sizes

## Reading data files

If we want to read data files, we can use the read/writer module provided by `simframe` that is used to write the data.

In [ ]:
from dustpy import hdf5writer

wrtr = hdf5writer()

We should make sure that the correct data directory is assigned to the writer.

In [ ]:
wrtr

We can now read a single data file with

In [ ]:
data0003 = wrtr.read.output(3)

This function returns a namespace and the data can simply be accessed in the same way as for the `Simulation` object.

In [ ]:
data0003.gas.Sigma

We can also read the entire data directory with

In [ ]:
data = wrtr.read.all()

The data has now an additional dimension for time.

In [ ]:
data.gas.Sigma.shape

Data files can be quite large and reading the entire data set can consequently take some time. Instead, it is more efficient to only read single fields from the data files. We can do so via e.g.

In [ ]:
SigmaGas = wrtr.read.sequence("gas.Sigma")

In [ ]:
SigmaGas.shape

It is also possible to exclude certain fields from being written into the data files to save memory by setting their `save` attribute to `False`.

In [ ]:
sim.dust.v.save = False

## Reading Dump Files

The data files merely contain the pure data, but no information about the operations `TriPodPy` has to perform, e.g. customized functions. Hence, it is not possible to directly restart a simulation from data files.

`simframe` is saving by default the most recent dump file, from which a simulation can be restarted.

**Attention:** Malware can be injected with dump files, which are pickled objects. One should only read dump files that were created by oneself or from trusted sources! Dump files have to be read with the same version of `TriPodPy` as they were written. Otherwise, it is not guaranteed to work.

In [ ]:
from dustpy import readdump

In [ ]:
sim_restart = readdump("data/frame.dmp") 

This is now a simulation frame that should be identical to our previous object.

In [ ]:
sim.gas.Sigma == sim_restart.gas.Sigma

We can now, for example, add more snapshots and restart the simulation. Here we just want to extend the run by one year.

In [ ]:
from dustpy import constants as c

sim_restart.t.snapshots = np.concatenate((sim_restart.t.snapshots, [100001.*c.year]))

The current time is

In [ ]:
sim_restart.t / c.year

We can now restart the simulation for another year after the time of the dumpfile (which is automatically used).

In [ ]:
sim_restart.run()

Another file was written and the current time is now

In [ ]:
sim_restart.t / c.year